In [1]:
import numpy as np
from math import floor, ceil
from random import randint, choice
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
from tqdm import tqdm
from tqdm.autonotebook import tqdm
import time as tm
import multiprocessing as mp   
from LAb_UW_functions import *

/tmp/ipykernel_15165/1597998577.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
###### INPUT #######

machine_name = "on_bench"
experiment_name = "glued_pzt"
data_type = "data_tsv_files"

### Some plot features ###
image_types = ("waveforms","amplitude_maps","amplitude_spectrum_maps", "example_of_filtered_spectrum","example_of_filtered_waveform")
formato = '.png'
remove_initial_samples = 50    # number of samples to be removed at the beginning, to get rid of the noise burst.
highlight_start = 0
highlight_end = 0
ylim_plot = 500
xlim_plot= 20
ticks_steps_waveforms = 1 #[microseconds] plot ticks 
step_wf_to_plot = 10  # get one waveforms each step_wf_to_plot 
amp_scale = 300  #color map for amplitude scale
freq_cut = 5   # [Hz] lowpass frequency threshold 


infile_path_list = make_infile_path_list(machine_name,experiment_name,  data_type = data_type)
outdir_path_images = make_images_folders(machine_name, experiment_name, image_types)       

for infile_path in infile_path_list:
    print('PROCESSING UW DATA IN %s: ' %infile_path)

    data,metadata = make_UW_data(infile_path)
    data,metadata = remove_starting_noise(data, metadata, remove_initial_samples)

    #CHOOSE THE FILE PATH OF THE OUTPUT. Add here a line for each type of images you want and call the relative plot function with the choosen path
    outfile_name = os.path.basename(infile_path).split('.')[0]    # remove extension

    outfile_path_waveforms = os.path.join(outdir_path_images[0], outfile_name)
    outfile_path_amp_map = os.path.join(outdir_path_images[1], outfile_name)
    outfile_path_spectrum_map = os.path.join(outdir_path_images[2], outfile_name)
    outfile_path_example_filtered_spectrum = os.path.join(outdir_path_images[3], outfile_name)
    outfile_path_example_original_vs_filtered = os.path.join(outdir_path_images[4], outfile_name)


    ## PLOT DATA ###
    start_time = tm.time()
    uw_all_plot(data, metadata, step_wf_to_plot, highlight_start, highlight_end, 
                        ylim_plot, xlim_plot, ticks_steps_waveforms, outfile_path = outfile_path_waveforms, formato = formato)
    amplitude_map(data,metadata, amp_scale, outfile_path=outfile_path_amp_map, formato = formato)

    # FILTER WITH A LOWPASS AND PLOT THE AMPLITUDE MAP OF THE SPECTRUM
    filtered_data, noise_recostructed = signal2noise_separation_lowpass(data, metadata, freq_cut=freq_cut, outfile_path= outfile_path_spectrum_map, formato=formato )

    # PLOT THE FILTERED DATA IN THE SAME FOLDERS OF THE ORIGINAL DATA
    uw_all_plot(filtered_data, metadata, step_wf_to_plot, highlight_start, highlight_end, 
                        ylim_plot, xlim_plot, ticks_steps_waveforms, outfile_path = outfile_path_waveforms + "_filtered", formato = formato)
    amplitude_map(filtered_data,metadata, amp_scale,outfile_path = outfile_path_amp_map + "_filtered", formato = formato)

    wave_chosed = randint(0,metadata['number_of_waveforms'])
    single_waveform = data[wave_chosed,:]
    filtered_single_waveform, filtered_noise_recostructed = signal2noise_separation_lowpass(single_waveform, metadata, freq_cut=freq_cut, outfile_path=outfile_path_example_filtered_spectrum)
    signal_vs_filtered_signal_plot(single_waveform, filtered_single_waveform, metadata, freq_cut=freq_cut, outfile_path = outfile_path_example_original_vs_filtered, )


    print("--- %s seconds for processing %s---" % (tm.time() - start_time, outfile_name))

        

PROCESSING UW DATA IN /home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/data_tsv_files/PIS1_PIS2_glued_1000ns.bscan.tsv: 
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/waveforms/PIS1_PIS2_glued_1000ns
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/amplitude_maps/PIS1_PIS2_glued_1000ns
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/amplitude_spectrum_maps/PIS1_PIS2_glued_1000ns
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/waveforms/PIS1_PIS2_glued_1000ns_filtered
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/amplitude_maps/PIS1_PIS2_glued_1000ns_filtered
Plot saved to /home/michele/Desktop/Dottorato/active_source_impl

In [3]:
# USE MULTIPROCESSING INSTEAD OF FOR CYCLE ABOVE
# # Step 1: Init multiprocessing.Pool()
# pool = mp.Pool(mp.cpu_count())    # use all available cores

# # Step 2: `pool.apply` calcolo_pdf
# x = pool.map_async(make_image, [waveform for waveform in infile_path_list]) # apply to all the waveforms not already computed
# x.get()

# # Step 3: Don't forget to close
# pool.close()



In [4]:
# # Is the noise gaussian?
# # Fit a normal distribution to the data:

# # mean and standard deviation
# data2hist = noise_recostructed
# mu, std = norm.fit(data2hist) 
# mu1, std1 = norm.fit(data2hist) 
 
# # Plot the histogram as an experimental PDF
# # plt.hist(data2hist, bins=100, density=True, alpha=0.6, color='b')
# counts, bins = np.histogram(noise_recostructed, bins=100)
# plt.hist(bins[:-1], bins, weights=counts, density=True, alpha=0.6, color="b")

# min_data = np.amin(data2hist)
# max_data= np.amax(data2hist)
# plt.xlim(min_data,max_data)

# # Plot the theoretical PDF
# xmin, xmax = plt.xlim()
# x = np.linspace(xmin, xmax, 100)
# p = norm.pdf(x, mu, std)
# p1 = norm.pdf(x, mu1, std1)
 
# plt.plot(x, p, 'k', linewidth=2)
# plt.plot(x, p1, 'r', linewidth=2)
# title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)
# plt.title(title)
 
# plt.show()

In [5]:
outdir_path_images

['/home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/waveforms',
 '/home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/amplitude_maps',
 '/home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/amplitude_spectrum_maps',
 '/home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/example_of_filtered_spectrum',
 '/home/michele/Desktop/Dottorato/active_source_implementation/experiments_on_bench/glued_pzt/images/example_of_filtered_waveform']